In [ ]:
pip install lightgbm

In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate
from pathlib import Path
from cProfile import label
from jinja2 import Undefined
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import metrics,tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from datamining_machine import DataMiningMachine
from sklearn.model_selection import train_test_split

In [19]:
test_data  = pd.read_csv("test_set_VU_DM.csv")

In [2]:
training_data  = pd.read_csv("training_set_VU_DM.csv")

In [3]:
aux_training = training_data

In [38]:
len(test_data)

4959183

In [4]:
#create new column for NDCG evaluation

aux_training['target_col'] = aux_training.apply(
    lambda row: (5 if row.booking_bool == 1 else (1 if row.click_bool == 1 else 0)), axis=1)

In [5]:
print("len target 0: ",len(aux_training.loc[aux_training["target_col"] == 0]))
print("len target 1: ",len(aux_training.loc[aux_training["target_col"] == 1]))
print("len target 5: ",len(aux_training.loc[aux_training["target_col"] == 5]))


len target 0:  4736468
len target 1:  83489
len target 5:  138390


In [7]:
## get set balanced for target column
np.random.seed(0)
balanced = aux_training.groupby('target_col')
balanced = balanced.apply(lambda x: x.sample(balanced.size().min()))

print("len target 0: ",len(balanced.loc[balanced["target_col"] == 0]))
print("len target 1: ",len(balanced.loc[balanced["target_col"] == 1]))
print("len target 5: ",len(balanced.loc[balanced["target_col"] == 5]))

len target 0:  83489
len target 1:  83489
len target 5:  83489


In [ ]:
## get set balanced for booking bool column
#np.random.seed(0)
#balanced = training_data.groupby('booking_bool')
#balanced = balanced.apply(lambda x: x.sample(balanced.size().min()))

print("len booking 0: ",len(balanced.loc[balanced["booking_bool"] == 0]))
print("len booking 1: ",len(balanced.loc[balanced["booking_bool"] == 1]))

In [ ]:
#np.random.seed(0)
#rand_sample2 = training_data.groupby('booking_bool').sample(frac=.5) #random
#rand_sample2.head(50)

In [ ]:
# np.random.seed(0)
#rand_sample = training_data.groupby('srch_id').sample(n=5) #random sample keeping 5 rows for any srch_id

## SAMPLE KEEPING SRCH_IDs OR BOOKING BOOL?

In [8]:
aux_training = balanced

In [ ]:
# aux_training = training_data.head(100)

#get a random sample of training data... what tho do about unbalanced data ????
#aux_training = training_data.sample(n=15000) 
#print(aux_training.head())


In [9]:
#drop columns i dont care or have too many empty values:
aux_training = aux_training.drop('date_time', axis=1)

aux_training = aux_training.drop('comp1_rate', axis=1)
aux_training = aux_training.drop('comp1_inv', axis=1)
aux_training = aux_training.drop('comp1_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp2_rate', axis=1)
aux_training = aux_training.drop('comp2_inv', axis=1)
aux_training = aux_training.drop('comp2_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp3_rate', axis=1)
aux_training = aux_training.drop('comp3_inv', axis=1)
aux_training = aux_training.drop('comp3_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp4_rate', axis=1)
aux_training = aux_training.drop('comp4_inv', axis=1)
aux_training = aux_training.drop('comp4_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp5_rate', axis=1)
aux_training = aux_training.drop('comp5_inv', axis=1)
aux_training = aux_training.drop('comp5_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp6_rate', axis=1)
aux_training = aux_training.drop('comp6_inv', axis=1)
aux_training = aux_training.drop('comp6_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp7_rate', axis=1)
aux_training = aux_training.drop('comp7_inv', axis=1)
aux_training = aux_training.drop('comp7_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp8_rate', axis=1)
aux_training = aux_training.drop('comp8_inv', axis=1)
aux_training = aux_training.drop('comp8_rate_percent_diff', axis=1)

aux_training = aux_training.drop('visitor_hist_starrating', axis=1)
aux_training = aux_training.drop('visitor_hist_adr_usd', axis=1)

aux_training = aux_training.drop('srch_query_affinity_score', axis=1)

aux_training = aux_training.drop('gross_bookings_usd', axis=1)

aux_training = aux_training.drop('prop_review_score', axis=1)
aux_training = aux_training.drop('prop_location_score2', axis=1)
aux_training = aux_training.drop('orig_destination_distance', axis=1)

In [10]:
##  drop columns that are not in test set (??? ) 
aux_training = aux_training.drop('click_bool', axis=1)   #replaced by target_col (?)
aux_training = aux_training.drop('booking_bool', axis=1) #replaced by target_col (?)
aux_training = aux_training.drop('position', axis=1)     # not sure about drop this one. Maybe some transformation ??

In [11]:
aux_training.head()

srch_id  site_id  visitor_location_country_id  \
target_col                                                          
0          382595     25637        5                          219   
           1863462   125067       14                          100   
           2167918   145543        9                           59   
           668300     44703        5                          219   
           1127991    75323        5                          219   

                    prop_country_id  prop_id  prop_starrating  \
target_col                                                      
0          382595               219   125378                3   
           1863462              219    21688                3   
           2167918               59    83244                4   
           668300               219   132563                3   
           1127991              219     1495                2   

                    prop_brand_bool  prop_location_score1  \
target_col                                                  
0          382595                 1                  1.95   
           1863462                0                  3.40   
           2167918                1                  0.00   
           668300                 1                  3.66   
           1127991                1                  3.26   

                    prop_log_historical_price  price_usd  promotion_flag  \
target_col                                                                 
0          382595                        4.92     124.00               0   
           1863462                       0.00     153.77               0   
           2167918                       5.16     185.11               1   
           668300                        4.79     149.00               0   
           1127991                       4.36      33.00               1   

                    srch_destination_id  srch_length_of_stay  \
target_col                                                     
0          382595                 10518                    4   
           1863462                 4748                    2   
           2167918                27000                    1   
           668300                  6443                    2   
           1127991                26067                    1   

                    srch_booking_window  srch_adults_count  \
target_col                                                   
0          382595                    62                  2   
           1863462                    3                  4   
           2167918                    0                  2   
           668300                    34                  2   
           1127991                    0                  2   

                    srch_children_count  srch_room_count  \
target_col                                                 
0          382595                     0                1   
           1863462                    0                1   
           2167918                    0                1   
           668300                     0                1   
           1127991                    0                1   

                    srch_saturday_night_bool  random_bool  target_col  
target_col                                                             
0          382595                          0            1           0  
           1863462                         1            1           0  
           2167918                         1            1           0  
           668300                          1            0           0  
           1127991                         0            0           0

In [12]:
feature = aux_training.drop('target_col', axis=1)
target  =  aux_training['target_col']

In [13]:
print("len 0: ",len(aux_training.loc[aux_training["target_col"] == 0]))
print("len 1: ",len(aux_training.loc[aux_training["target_col"] == 1]))
print("len 5: ",len(aux_training.loc[aux_training["target_col"] == 5]))

len 0:  83489
len 1:  83489
len 5:  83489


In [14]:
print("Empty values per column: \n",aux_training.isnull().sum())

Empty values per column: 
 srch_id                        0
site_id                        0
visitor_location_country_id    0
prop_country_id                0
prop_id                        0
prop_starrating                0
prop_brand_bool                0
prop_location_score1           0
prop_log_historical_price      0
price_usd                      0
promotion_flag                 0
srch_destination_id            0
srch_length_of_stay            0
srch_booking_window            0
srch_adults_count              0
srch_children_count            0
srch_room_count                0
srch_saturday_night_bool       0
random_bool                    0
target_col                     0
dtype: int64


In [15]:
X_train, X_val, y_train, y_val = train_test_split(feature, target, test_size=0.2, random_state=1)

print("X_train.shape = ",X_train.shape)
print("X_val.shape = ",X_val.shape)
print("\ny_train.shape = ",y_train.shape)
print("y_val.shape = ",y_val.shape)

    

X_train.shape =  (200373, 19)
X_val.shape =  (50094, 19)

y_train.shape =  (200373,)
y_val.shape =  (50094,)


In [16]:
qids_train = X_train.groupby("srch_id")["srch_id"].count().to_numpy()
qids_val   = X_val.groupby("srch_id")["srch_id"].count().to_numpy()


In [17]:
print("query train ids mean: ",qids_train.mean()) # should we ensure there are enough rows from same srch_id ??? 
print("query val   ids mean: ",qids_val.mean()) # should we ensure there are enough rows from same srch_id ??? 

query train ids mean:  1.4039588004484305
query val   ids mean:  1.1022752277428156


In [ ]:
#X_train = X_train.drop('srch_id', axis=1) #do we need to drop search id?
#X_val   = X_val.drop('srch_id', axis=1)   #do we need to drop search id?

In [18]:
gbm = lgb.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
)

gbm.fit(X_train, 
        y_train, 
        group=qids_train,
        eval_set=[(X_val, y_val)], 
        eval_group=[qids_val],
        eval_at=[5, 10,20]
)

[1]	valid_0's ndcg@5: 0.990598	valid_0's ndcg@10: 0.990598	valid_0's ndcg@20: 0.990598
[2]	valid_0's ndcg@5: 0.990623	valid_0's ndcg@10: 0.990623	valid_0's ndcg@20: 0.990623
[3]	valid_0's ndcg@5: 0.990684	valid_0's ndcg@10: 0.990685	valid_0's ndcg@20: 0.990685
[4]	valid_0's ndcg@5: 0.99068	valid_0's ndcg@10: 0.990681	valid_0's ndcg@20: 0.990681
[5]	valid_0's ndcg@5: 0.990648	valid_0's ndcg@10: 0.990653	valid_0's ndcg@20: 0.990653
[6]	valid_0's ndcg@5: 0.990634	valid_0's ndcg@10: 0.99064	valid_0's ndcg@20: 0.99064
[7]	valid_0's ndcg@5: 0.990703	valid_0's ndcg@10: 0.990709	valid_0's ndcg@20: 0.990709
[8]	valid_0's ndcg@5: 0.990805	valid_0's ndcg@10: 0.99081	valid_0's ndcg@20: 0.99081
[9]	valid_0's ndcg@5: 0.990809	valid_0's ndcg@10: 0.990814	valid_0's ndcg@20: 0.990814
[10]	valid_0's ndcg@5: 0.990789	valid_0's ndcg@10: 0.990794	valid_0's ndcg@20: 0.990794
[11]	valid_0's ndcg@5: 0.990833	valid_0's ndcg@10: 0.990838	valid_0's ndcg@20: 0.990838
[12]	valid_0's ndcg@5: 0.990835	valid_0's ndcg

[100]	valid_0's ndcg@5: 0.99095	valid_0's ndcg@10: 0.990955	valid_0's ndcg@20: 0.990955


LGBMRanker(metric='ndcg', objective='lambdarank')

In [ ]:
#test_pred = gbm.predict(X_test)

#print(test_pred)

In [29]:
X_train.columns

Index(['srch_id', 'site_id', 'visitor_location_country_id', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_brand_bool', 'prop_location_score1',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'random_bool'],
      dtype='object')

In [53]:
aux_test = test_data

In [54]:
#drop columns i dont care or have too many empty values:


aux_test = aux_test.drop('date_time', axis=1)

aux_test = aux_test.drop('comp1_rate', axis=1)
aux_test = aux_test.drop('comp1_inv', axis=1)
aux_test = aux_test.drop('comp1_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp2_rate', axis=1)
aux_test = aux_test.drop('comp2_inv', axis=1)
aux_test = aux_test.drop('comp2_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp3_rate', axis=1)
aux_test = aux_test.drop('comp3_inv', axis=1)
aux_test = aux_test.drop('comp3_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp4_rate', axis=1)
aux_test = aux_test.drop('comp4_inv', axis=1)
aux_test = aux_test.drop('comp4_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp5_rate', axis=1)
aux_test = aux_test.drop('comp5_inv', axis=1)
aux_test = aux_test.drop('comp5_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp6_rate', axis=1)
aux_test = aux_test.drop('comp6_inv', axis=1)
aux_test = aux_test.drop('comp6_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp7_rate', axis=1)
aux_test = aux_test.drop('comp7_inv', axis=1)
aux_test = aux_test.drop('comp7_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp8_rate', axis=1)
aux_test = aux_test.drop('comp8_inv', axis=1)
aux_test = aux_test.drop('comp8_rate_percent_diff', axis=1)

aux_test = aux_test.drop('visitor_hist_starrating', axis=1)
aux_test = aux_test.drop('visitor_hist_adr_usd', axis=1)

aux_test = aux_test.drop('srch_query_affinity_score', axis=1)

#aux_test = aux_test.drop('gross_bookings_usd', axis=1)

aux_test = aux_test.drop('prop_review_score', axis=1)
aux_test = aux_test.drop('prop_location_score2', axis=1)
aux_test = aux_test.drop('orig_destination_distance', axis=1)

In [55]:
aux_test.columns

Index(['srch_id', 'site_id', 'visitor_location_country_id', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_brand_bool', 'prop_location_score1',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'random_bool'],
      dtype='object')

In [35]:
test_pred = gbm.predict(aux_test)

print(test_pred)

[ 0.22364107  0.2187913  -0.03791066 ...  0.45448734  0.4923028
  0.63498814]


In [56]:
aux_test["predicted_ranking"] = test_pred


In [57]:
aux_test.sort_values(["srch_id","predicted_ranking"], ascending=[True,False])

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_brand_bool,prop_location_score1,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,predicted_ranking
23,1,24,216,219,99484,3,1,2.40,4.54,69.00,1,19222,1,10,2,0,1,0,0,0.671554
9,1,24,216,219,54937,3,1,2.08,4.75,83.30,1,19222,1,10,2,0,1,0,0,0.641948
12,1,24,216,219,61934,3,1,2.89,4.69,88.88,1,19222,1,10,2,0,1,0,0,0.570841
8,1,24,216,219,50162,2,1,2.20,4.36,49.99,1,19222,1,10,2,0,1,0,0,0.483287
5,1,24,216,219,28181,3,1,2.30,4.68,84.15,0,19222,1,10,2,0,1,0,0,0.366157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959177,332787,24,216,117,29018,4,1,2.56,4.64,70.05,0,19246,2,7,1,0,1,0,0,0.509235
4959181,332787,24,216,117,94437,4,0,2.94,4.64,66.07,0,19246,2,7,1,0,1,0,0,0.492303
4959176,332787,24,216,117,22854,5,1,2.30,4.96,109.71,0,19246,2,7,1,0,1,0,0,0.483032
4959180,332787,24,216,117,35240,4,0,1.79,4.64,73.91,0,19246,2,7,1,0,1,0,0,0.454487


In [58]:
aux_test2 = aux_test
aux_test2


,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_brand_bool,prop_location_score1,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,predicted_ranking
0,1,24,216,219,3180,3,1,2.94,5.03,119.00,0,19222,1,10,2,0,1,0,0,0.223641
1,1,24,216,219,5543,3,1,2.64,4.93,118.00,0,19222,1,10,2,0,1,0,0,0.218791
2,1,24,216,219,14142,2,1,2.71,4.16,49.00,0,19222,1,10,2,0,1,0,0,-0.037911
3,1,24,216,219,22393,3,1,2.40,5.03,143.00,0,19222,1,10,2,0,1,0,0,0.090092
4,1,24,216,219,24194,3,1,2.94,4.72,79.00,0,19222,1,10,2,0,1,0,0,0.313188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959178,332787,24,216,117,32019,4,0,2.48,4.53,66.07,0,19246,2,7,1,0,1,0,0,0.409020
4959179,332787,24,216,117,33959,4,1,2.20,4.39,67.10,0,19246,2,7,1,0,1,0,0,0.566881
4959180,332787,24,216,117,35240,4,0,1.79,4.64,73.91,0,19246,2,7,1,0,1,0,0,0.454487
4959181,332787,24,216,117,94437,4,0,2.94,4.64,66.07,0,19246,2,7,1,0,1,0,0,0.492303


In [59]:
aux_test2.drop(aux_test2.columns.difference(['srch_id','prop_id']), 1, inplace=True)

In [60]:
aux_test2.head(50)

,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194
5,1,28181
6,1,34263
7,1,37567
8,1,50162
9,1,54937


In [61]:
aux_test2.to_csv("out.csv", index=False)    

# -------------------------------------------------------------------------------------------------------------